In [2]:
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
from skimage import morphology
import os
import random
from skimage.measure import label,regionprops

In [3]:
#root = "Z:/Projectline_modelling_lung_cancer/Nii_Processed/"
root = "//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/Users/Luis/CT_ITV_GTV_XBP_Nii_Processed/"

In [4]:
def LookFiles_Snapshot(folder_path):
    planCT_list,planCTLung_list = ([],[])
    bp50_list, bp50Lung_list = ([],[])
    gtv_list, itv_list = ([],[])
    for root,dirs,files in os.walk(folder_path):
        for f in files:
            if "PlanCT_CTProcessed.nii" in f:
                planCT_list.append(os.path.join(folder_path,f))
            if "PlanCT_LungProcessed.nii" in f:
                planCTLung_list.append(os.path.join(folder_path,f))
            if "bp50_CTProcessed.nii" in f:
                bp50_list.append(os.path.join(folder_path,f))
            if "bp50_LungProcessed.nii" in f:
                bp50Lung_list.append(os.path.join(folder_path,f))
            if "GTV" in f:
                gtv_list.append(os.path.join(folder_path,f))
            if "ITV" in f:
                itv_list.append(os.path.join(folder_path,f))
    
    #print("Len of Files",len(planCT_list),len(planCTLung_list),len(bp50_list),len(bp50Lung_list),len(gtv_list),len(itv_list))


    return planCT_list,planCTLung_list,bp50_list,bp50Lung_list,gtv_list,itv_list
    

In [5]:
PxList = os.listdir(root)
len(PxList)

172

In [7]:
def CountSlidesPerCategory(struct_np,tumorSlide_Tresh):
    #Z
    indices_Tum_z = []
    indices_Semi_z = []
    for i in range(struct_np.shape[-1]):
        if np.sum(struct_np[:,:,i])>0:
            currRatio = np.sum(struct_np[:,:,i])/(struct_np.shape[0]*struct_np.shape[1])
            if currRatio > tumorSlide_Tresh[-1]:
                indices_Tum_z.append(i)
            elif currRatio > 0:
                indices_Semi_z.append(i)
    #Y
    indices_Tum_y= []
    indices_Semi_y = []
    for i in range(struct_np.shape[-2]):
        if np.sum(struct_np[:,i,:])>0:
            currRatio = np.sum(struct_np[:,i,:])/(struct_np.shape[0]*struct_np.shape[2])
            if currRatio > tumorSlide_Tresh[-2]:
                indices_Tum_y.append(i)
            elif currRatio > 0:
                indices_Semi_y.append(i)
    #X
    indices_Tum_x= []
    indices_Semi_x = []
    for i in range(struct_np.shape[-3]):
        if np.sum(struct_np[i,:,:])>0:
            currRatio = np.sum(struct_np[i,:,:])/(struct_np.shape[1]*struct_np.shape[2])
            if currRatio > tumorSlide_Tresh[-3]:
                indices_Tum_x.append(i)
            elif currRatio > 0:
                indices_Semi_x.append(i)

    return indices_Tum_x,indices_Tum_y,indices_Tum_z,indices_Semi_x,indices_Semi_y,indices_Semi_z

In [8]:
def group_consecutive_numbers(lst):
    result = []
    sublist = [lst[0]]
    for i in range(1, len(lst)):
        if lst[i] - 1 == lst[i - 1]:
            sublist.append(lst[i])
        else:
            result.append(sublist)
            sublist = [lst[i]]
    result.append(sublist)
    return result

def adjust_sublists(lst):
    adjusted_result = []
    for sublist in lst:
        if len(sublist) > 1:
            adjusted_result.append([sublist[0], sublist[1], sublist[2]])
        else:
            adjusted_result.append([sublist[0] - 1, sublist[0], sublist[0] + 1])
    return adjusted_result





In [18]:
def GetTresholds():
    factor=.6
    avgRatio_Avg= [0.130966154,1.2537841,1.24953586]
    avgRatio_Med= [0.010234375,0.0100585938,0.0086767578]
    tumorSlide_AvgTresh = [avgRatio_Avg[0]*factor,avgRatio_Avg[1]*factor,avgRatio_Avg[2]*factor]
    tumorSlide_MediTresh = [avgRatio_Med[0]*factor,avgRatio_Med[1]*factor,avgRatio_Med[2]*factor]
    return tumorSlide_MediTresh

In [19]:
okPx,errorPx = [],[]

for Px in PxList:
    planCT_list,planCTLung_list,bp50_list,bp50Lung_list,gtv_list,itv_list = LookFiles_Snapshot(root+Px)
    flagOkPx = False

    if len(gtv_list)>0 and len(bp50_list)>0:
        tum_np  = nib.load(gtv_list[0]).get_fdata()
        flagOkPx = True
    elif len(itv_list)>0 and len(planCT_list)>0:
        tum_np  = nib.load(itv_list[0]).get_fdata()
        flagOkPx = True
    else:
        errorPx.append(Px)
        
    if flagOkPx:
        tum_filled = tum_np
        if np.sum(tum_filled)!=0:
            okPx.append(Px)

            tumorSlide_MediTresh = GetTresholds()

            indices_Tum_x,indices_Tum_y,indices_Tum_z,indices_Semi_x,indices_Semi_y,indices_Semi_z= CountSlidesPerCategory(tum_filled,tumorSlide_MediTresh)

            # Group consecutive numbers
            grouped_list = group_consecutive_numbers(indices_Tum_x)

            # Adjust sublists
            adjusted_list = adjust_sublists(grouped_list)

            print(indices_Tum_x)
            print(grouped_list)
            print(adjusted_list)

            break


[142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208]
[[142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208]]
[[142, 143, 144]]


In [15]:
print(indices_Tum_y)


[36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126]


In [32]:
 if False and len(gtvTot)>0:
    tum_np  = nib.load(gtvTot[0]).get_fdata()
    ct_np  = nib.load(bp50[0]).get_fdata()
    ContourType="GTV"
else:
    tum_np  = nib.load(itvTot[0]).get_fdata()
    ct_np  = nib.load(planct_path[0]).get_fdata()
    print(nib.load(itvTot[0]).affine)
    print(nib.load(planct_path[0]).affine)
    ContourType="ITV"
print(tum_np.shape,ct_np.shape)

[[  -0.9765625    -0.            0.          249.51171875]
 [  -0.           -0.9765625     0.          427.01171875]
 [   0.            0.            2.         -150.        ]
 [   0.            0.            0.            1.        ]]
[[  -0.9765625    -0.            0.          249.51171875]
 [  -0.           -0.9765625     0.          427.01171875]
 [   0.            0.            2.         -150.        ]
 [   0.            0.            0.            1.        ]]
(512, 512, 310) (512, 512, 155)


In [24]:
def getMiddleSlices(gtv_np):
    #Get mid tumor Slices per view
    non_zero_elements = np.nonzero(gtv_np)
    min_x, max_x = np.min(non_zero_elements[0]), np.max(non_zero_elements[0])
    middle_x = (min_x + max_x) // 2
    min_y, max_y = np.min(non_zero_elements[1]), np.max(non_zero_elements[1])
    middle_y = (min_y + max_y) // 2
    min_z, max_z = np.min(non_zero_elements[2]), np.max(non_zero_elements[2])
    middle_z = (min_z + max_z) // 2
    return middle_x,middle_y,middle_z
 
middle_x,middle_y,middle_z = getMiddleSlices(tum_np)

In [29]:
def Screenshot(ct_np,gtv_np,middle_x,middle_y,middle_z,contourBool=True,Px2Save=None,ImageType=None):
    minHU = -1024
    maxHU = 600
    savebool = True
    root2save= "Z:/Projectline_modelling_lung_cancer/JPEGs/Raw/"
    if Px2Save is not None:
        pathToSave = os.path.join(root2save,Px2Save)
        if not(os.path.exists(pathToSave)):
            os.mkdir(pathToSave)
    
    print(middle_x,middle_y,middle_z,Px2Save,ct_np.shape,ImageType,gtv_np.shape)

    
    clipped_image = np.clip(ct_np, minHU, maxHU)
    #middle_x,middle_y,middle_z = getMiddleSlices(gtv_np)
    plt.figure(figsize=(15, 15), dpi=300)
    plt.imshow(clipped_image[:,:,middle_//2],cmap='gray'),plt.axis('off')#,plt.colorbar()
    plt.title("Z:"+str(middle_z))
    if contourBool:
        plt.contour(gtv_np[:,:,middle_z],linewidths=0.5,colors='r')
    if not(savebool): 
        plt.show()
    else:
        plt.savefig(pathToSave+"/"+ImageType+"_Axial.jpeg")
        plt.close()
    plt.figure(figsize=(15, 15), dpi=300)
    plt.imshow(np.rot90(clipped_image[:,middle_y//2,:]),cmap='gray'),plt.axis('off')#,plt.colorbar()
    plt.title("Y:"+str(middle_y))
    if contourBool:
        plt.contour(np.rot90(gtv_np[:,middle_y,:]),linewidths=0.5,colors='r')
    if not(savebool): 
        plt.show()
    else:
        plt.savefig(pathToSave+"/"+ImageType+"_Sagital.jpeg")
        plt.close()
    plt.figure(figsize=(15, 15), dpi=300)
    plt.imshow(np.rot90(clipped_image[middle_x//2,:,:]),cmap='gray'),plt.axis('off')#,plt.colorbar()
    plt.title("X:"+str(middle_x))
    if contourBool:
        plt.contour(np.rot90(gtv_np[middle_x,:,:]),linewidths=0.5,colors='r')
    if not(savebool): 
        plt.show()
    else:
        plt.savefig(pathToSave+"/"+ImageType+"_Coronal.jpeg")
        plt.close()

Screenshot(ct_np,tum_np,middle_x,middle_y,middle_z,True,Px,ContourType+"Raw")


271 216 163 1007981 (512, 512, 155) ITVRaw (512, 512, 310)


NameError: name 'middle_' is not defined

<Figure size 4500x4500 with 0 Axes>

In [42]:
indices_Semi_z

-1

In [43]:
indices_Tum_z

-1